Reprojection
- Reproject to EPSG:4326
- Ensure pre-disaster raster and post-disaster raster have the same resolution (30 cm)
- Crop raster to removed masked pixels as much as possible
- Compress raster using DEFLATE

In [12]:
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject
from rasterio.enums import Resampling
from rasterio.windows import Window

def reproject_and_crop_raster(raster_path, output_path, target_crs='EPSG:4326', resampling_method=Resampling.bilinear):
    """
    Reproject a raster to a specified target CRS, crop to remove masked pixels, and compress the output.

    Parameters:
        raster_path (str): Path to the input raster file.
        target_crs (str): The target CRS in 'EPSG:xxxx' format.
        resampling_method (Resampling): Resampling method from rasterio.enums.Resampling.

    Returns:
        output_path: Path to the saved reprojected and cropped raster.
    """
    
    with rasterio.open(raster_path) as raster:
        # 1. Reprojection
        transform, width, height = calculate_default_transform(
            raster.crs, target_crs, raster.width, raster.height, *raster.bounds,resolution=(3.061967147110499275e-06, 3.061967147110499275e-06))
        
        reprojected_data = np.empty((raster.count, height, width), dtype=raster.dtypes[0])
        reproject(
            source=rasterio.band(raster, list(range(1, raster.count+1))),
            destination=reprojected_data,
            src_transform=raster.transform,
            src_crs=raster.crs,
            dst_transform=transform,
            dst_crs=target_crs,
            resampling=resampling_method)

        # 2. Cropping to remove masked pixels
        no_data_value = 0 # Modify this if your nodata value is different
        rows, cols = np.where(reprojected_data[0] != no_data_value)
        min_row, max_row = rows.min(), rows.max() + 1
        min_col, max_col = cols.min(), cols.max() + 1
        window = Window(col_off=min_col, row_off=min_row, width=max_col-min_col, height=max_row-min_row)
        cropped_data = reprojected_data[:, min_row:max_row, min_col:max_col]

        # Update the transform for the cropped raster
        cropped_transform = rasterio.windows.transform(window, transform)

        # 3. Compression and save to output TIFF
        profile = raster.profile.copy()
        profile.update({
            'crs': target_crs,
            'transform': cropped_transform,
            'width': window.width,
            'height': window.height,
            'compress': 'deflate'  # Using deflate compression
        })
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(cropped_data)

    return output_path

In [13]:
import os

img_dir = 'turkey_earthquake/images/merged_data/'
output_dir = 'turkey_earthquake/images/reprojected_data/'
img_list = [file for file in os.listdir(img_dir) if file.endswith('.tif') or file.endswith('.tiff')]
img_list

['Osmaniye_20221227_10300100DF069700_pre_disaster.tif',
 'Osmaniye_20230208_10300500D9F8D400_post_disaster.tif']

In [15]:
%%time
input_path = img_dir+img_list[1]
output_path = output_dir+img_list[1]
output_path = reproject_and_crop_raster(input_path, output_path, target_crs='EPSG:4326')
output_path

CPU times: user 2min 15s, sys: 15.8 s, total: 2min 31s
Wall time: 2min 41s


'turkey_earthquake/images/reprojected_data/Osmaniye_20230208_10300500D9F8D400_post_disaster.tif'

## Script to run the whole dataset

In [ ]:
from tqdm import tqdm

img_dir = 'turkey_earthquake/images/merged_data/'
output_dir = 'turkey_earthquake/images/reprojected_data/'
img_list = [file for file in os.listdir(img_dir) if file.endswith('.tif') or file.endswith('.tiff')]

for i in tqdm(range(len(img_list))):
    input_path = img_dir+img_list[i]
    output_path = output_dir+img_list[i]
    reproject_and_crop_raster(input_path, output_path, target_crs='EPSG:4326')